In [1]:
import pandas as pd
dados = pd.read_csv('medicoes_jlle.csv')
dados.head()

,Chuva Dia Anterior,Maré,Chuva,Temperatura,Resultado
0,1.000000,0.684211,1.000000,0.621118,1
1,0.000000,0.842105,0.380105,0.745342,1
2,0.259465,0.684211,0.196118,1.000000,1
3,0.000000,0.631579,0.268500,0.832298,1
4,0.229399,0.736842,0.219976,0.841615,1


In [2]:
X = dados.drop(['Resultado'],axis=1).values
y = dados['Resultado'].values
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0])

In [5]:
from sklearn.model_selection import train_test_split
X_treino,X_teste,y_treino,y_teste = train_test_split(X,y,test_size=0.2)

type(X_treino)
print(X_treino)

[[0.00000000e+00 7.36842105e-01 3.39668420e-02 6.83229814e-01]
 [3.45211580e-02 8.94736842e-01 4.08410840e-02 8.88198758e-01]
 [1.20267261e-01 5.78947368e-01 6.34856450e-02 1.05900621e+00]
 [2.52783964e-01 7.89473684e-01 5.41852000e-02 5.52795031e-01]
 [7.57238310e-02 5.78947368e-01 4.44803882e-01 8.94409938e-01]
 [0.00000000e+00 8.42105263e-01 3.80105135e-01 7.45341615e-01]
 [6.01336300e-02 6.31578947e-01 4.04367170e-02 6.89440994e-01]
 [4.32071269e-01 6.84210526e-01 2.99231700e-02 3.91304348e-01]
 [0.00000000e+00 7.89473684e-01 0.00000000e+00 9.13043478e-01]
 [0.00000000e+00 8.94736842e-01 2.70926001e-01 6.21118012e-01]
 [5.67928730e-02 6.31578947e-01 8.08734300e-03 8.07453416e-01]
 [0.00000000e+00 8.42105263e-01 0.00000000e+00 6.89440994e-01]
 [7.46102450e-02 7.89473684e-01 6.91467850e-02 7.51552795e-01]
 [8.90868600e-03 8.42105263e-01 5.78245050e-02 8.07453416e-01]
 [0.00000000e+00 6.31578947e-01 4.85240600e-03 7.82608696e-01]
 [0.00000000e+00 7.36842105e-01 9.42175500e-02 5.279503

In [4]:
import torch
X_treino = torch.FloatTensor(X_treino)
X_teste = torch.FloatTensor(X_teste)
y_treino = torch.LongTensor(y_treino)
y_teste = torch.LongTensor(y_teste)

print(y_teste)

tensor([1, 0, 0, 1, 1, 0, 0, 0, 1, 0])


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# class Modelo(nn.Module):
#   def __init__(self,entrada=4,camada_escondida1=8,camada_escondida2=32,saida=2):
#     super().__init__()
#     self.fc1 = nn.Linear(entrada,camada_escondida1)
#     self.fc2 = nn.Linear(camada_escondida1, camada_escondida2)
#     self.out = nn.Linear(camada_escondida2, saida)
  
#   def forward(self, x):
#     x = F.relu(self.fc1(x))
#     x = F.relu(self.fc2(x))
#     x = self.out(x)
#     return x

class Modelo(nn.Module):
  def __init__(self,entrada=4,camada_escondida1=8,camada_escondida2=32,saida=2):
    super().__init__()
    self.fc1 = nn.Linear(entrada,camada_escondida1)
    self.fc2 = nn.Linear(camada_escondida1, camada_escondida2)
    self.out = nn.Linear(camada_escondida2, saida)
  
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.out(x)
    return x

In [ ]:
modelo_classificacao = Modelo()

In [ ]:
funcao_objetivo = nn.CrossEntropyLoss()

In [ ]:
otimizador = torch.optim.Adam(modelo_classificacao.parameters(), lr=0.01)

In [ ]:
epocas = 100
custos = []
for i in range(epocas):
  y_predito = modelo_classificacao.forward(X_treino)
  custo = funcao_objetivo(y_predito, y_treino)
  custos.append(custo)

  otimizador.zero_grad()
  custo.backward()
  otimizador.step()

In [ ]:
preds = []
resul = []

with torch.no_grad():
    for val in X_teste:
        y_predito = modelo_classificacao.forward(val)
        print(y_predito)
        preds.append(y_predito.argmax().item())
        resul.append(y_predito)

df = pd.DataFrame({'Y': y_teste, 'YHat': preds})
df['Correto'] = [1 if corr == pred else 0 for corr, pred in zip(df['Y'], df['YHat'])]
df

tensor([ 0.7666, -0.6425])
tensor([ 2.4206, -2.0753])
tensor([ 1.9732, -1.6844])
tensor([ 3.7442, -3.2217])
tensor([ 3.3464, -2.8738])
tensor([ 3.0221, -2.5910])
tensor([-3.3039,  2.9136])
tensor([-3.0634,  2.6899])
tensor([ 1.5439, -1.3190])
tensor([ 3.9035, -3.3558])


,Y,YHat,Correto
0,0,0,1
1,0,0,1
2,0,0,1
3,0,0,1
4,0,0,1
5,0,0,1
6,1,1,1
7,1,1,1
8,0,0,1
9,0,0,1
